In [ ]:
import pandas as pd
import numpy as np
import time
import keplergl as kgl

# note that a geojson LineString and a shapely LineString are two different data types
from geojson import LineString as geoLS
from shapely.geometry import LineString as shapLS
from shapely.ops import unary_union
from shapely import MultiPoint

#take it from https://github.com/DKSR-Data-Competence-for-Cities-Regions/Urban-Data-Analytics/tree/dummy_data_generator/dksr_lib
from dksr import extract_sample_network
from dksr import trace_transform

In [ ]:
# a nice area in Kreuzberg
# a good adress to find your bbox of choice, make sure your bbox is not to small to achieve a nicer output
north, east, south, west = 52.460298,13.328991,52.515803,13.455849

In [ ]:
df = extract_sample_network(north, east, south, west, 50, seed=42)

In [ ]:
df.head(10)#.style.background_gradient(cmap='Blues', subset='length_km')

In [ ]:
df = trace_transform(df=df, speed=15, distance_delta=0.0001)

In [ ]:
df.head(10)

In [ ]:
"""
KeplerGL expects a GeoJSON string with 4 coordinates in order to visualize trips:
1) lng
2) lat
3) alt
4) timestamp
"""

# Create a new dataframe to store the resulting GeoJSON strings
geo_list = pd.DataFrame(np.zeros((len(df)),dtype=object),columns=['geo_json'])

# Iterate over each row in the input dataframe
for i in range(0,len(df)):
    
    # Create a list of zeros with the same length as the coordinates list for the current row
    z_list = [0] * len(df['coordinates'][i])
    
    # Combine the coordinates list with the zero list to create a new list with a third dimension of zeros
    list0 = df['coordinates'][i]
    list1 = np.insert(list0,2,z_list,axis=1)
    
    # Insert the timestamp list into the new list as the fourth dimension
    list2 = np.insert(list1,3,df['timestamps_list'][i],axis=1)
        
    # Convert the resulting 3D list to a GeoJSON string and store it in the new dataframe
    geo_list.iloc[i] = [geoLS(list2.tolist())]

In [ ]:
#OSMnx returns values in lat, lng while KeplerGL expetcs them in lng,lat
def switch_lat_lon(coords):
    return [coords[1], coords[0], coords[2], coords[3]]

In [ ]:
geo_list['geo_json'] = geo_list['geo_json'].apply(lambda x: {'type': x['type'], 'coordinates': [switch_lat_lon(coord) for coord in x['coordinates']]})

In [ ]:
#plot it
map = kgl.KeplerGl(height=800,data={'Scooters': geo_list})
map